![](https://europe-west1-atp-views-tracker.cloudfunctions.net/working-analytics?notebook=tutorials--agent-security-with-llamafirewall--output-guardrail)

# エージェントのガードレール：出力検証

## はじめに

AIエージェントをより安全にしたいと思ったことはありませんか？このチュートリアルでは、LlamaFirewallを使用して、有害または不整合な応答からエージェントを保護する出力検証ガードレールを構築します。

不整合は、AIエージェントの応答が意図された目的や指示から逸脱したときに発生します。例えば、カスタマーサービス用に設計されたエージェントが、金融アドバイスを提供したり、不適切なジョークを言い始めたりすると、それは不整合な動作と見なされます。不整合は、軽微な逸脱から、ビジネスやユーザーに損害を与える可能性のある有害な出力まで多岐にわたります。

**学習内容：**
- ガードレールとは何か、そしてなぜエージェントセキュリティに不可欠なのか
- LlamaFirewallを使用した出力検証の実装方法

出力検証の基本的なアーキテクチャを理解しましょう：

![出力ガードレール](assets/output-guardrail.png)

ここでは、`LlamaFirewall`がLLMの応答と元のユーザー入力の両方を受け取ることがわかります。両方のパラメータがアライメントチェックに使用されます。

## ガードレールについて

ガードレールはエージェントと並行して実行され、ユーザー入力のチェックと検証を可能にします。例えば、顧客のリクエストを支援するために非常にスマートな（そのため遅い/高価な）モデルを使用するエージェントがあるとします。悪意のあるユーザーに数学の宿題を手伝ってもらうようモデルに頼まれたくないでしょう。そこで、高速/安価なモデルでガードレールを実行できます。ガードレールが悪意のある使用を検出した場合、すぐにエラーを発生させ、高価なモデルの実行を停止して時間とお金を節約できます。

ガードレールには2種類あります：
1. 入力ガードレールは初期のユーザー入力で実行されます
2. 出力ガードレールは最終的なエージェント出力で実行されます

*このセクションは[OpenAI Agents SDKドキュメント](https://openai.github.io/openai-agents-python/guardrails/)から引用しています*

## 実装プロセス
 
`.env`ファイルに`TOGETHER_API_KEY`と`OPENAI_API_KEY`が含まれていることを確認してください

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # 現在のディレクトリで.envを探します

# TOGETHER_API_KEYが設定されているか確認（AlignmentCheckScannerに必要）
if not os.environ.get("TOGETHER_API_KEY"):
    print(
        "TOGETHER_API_KEY環境変数が設定されていません。このデモを実行する前に設定してください。"
    )
    exit(1)
else:   
    print ("TOGETHER_API_KEYが設定されています")

# OPENAI_API_KEYが設定されているか確認（エージェントに必要）
if not os.environ.get("OPENAI_API_KEY"):
    print(
        "OPENAI_API_KEY環境変数が設定されていません。このデモを実行する前に設定してください。"
    )
    exit(1)
else:
    print ("OPENAI_API_KEYが設定されています")

まず、ネストされた非同期サポートを有効にする必要があります。これにより、同期コードブロック内で非同期コードを実行できるようになり、一部のLlamaFirewall操作で必要となります。

In [ ]:
import nest_asyncio

# ネストされたイベントループを許可するためにnest_asyncioを適用
nest_asyncio.apply()

LlamaFirewallを初期化し、`ScannerType.AGENT_ALIGNMENT`を定義します

In [ ]:
from llamafirewall import (
    LlamaFirewall,
    Trace,
    Role,
    ScanDecision,
    ScannerType,
    UserMessage,
    AssistantMessage
)

# Prompt GuardとAlignment Checkerの両方のスキャナーでLlamaFirewallを初期化
lf = LlamaFirewall(
    scanners={
        Role.ASSISTANT: [ScannerType.AGENT_ALIGNMENT],
    }
)

便宜上、`LlamaFirewallOutput`を定義します

In [5]:
from pydantic import BaseModel

class LlamaFirewallOutput(BaseModel):
    is_harmful: bool
    score: float
    decision: str
    reasoning: str

次に、エージェントからのすべての応答に対して呼び出される`@output_guardrail`を定義します。
 
この例の`ctx`（コンテキスト）変数には最後のユーザー入力が含まれており、アライメントチェックのためのコンテキストを提供するのに役立ちます。

```python
user_input = ctx.context.get("user_input")
```
 
スキャンのために、最後のメッセージとエージェントの応答を含む`Trace`リストを作成します。トレースを`scan_replay`に送信します。これはLlamaFirewallがアライメントチェックのために提供するものです。
 ```python
    # アライメントチェックのための入力と出力メッセージのトレースを作成
     last_trace: Trace = [
         UserMessage(content=user_input),
         AssistantMessage(content=output)
     ]
 
     # LlamaFirewallのアライメントチェッカーを使用して出力をスキャン
     result = lf.scan_replay(last_trace)
 ```
**注意**: 完全な会話履歴やシステムプロンプトなど、より多くのコンテキストがあれば、モデルが会話の完全なコンテキストと意図をよりよく理解できるため、さらに良いアライメントチェックが可能になります。

In [ ]:
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    output_guardrail,
)

@output_guardrail
def llamafirewall_check_output(
    ctx: RunContextWrapper[None],
    agent: Agent,
    output: str
) -> GuardrailFunctionOutput:

    user_input = ctx.context.get("user_input")

    # アライメントチェックのための入力と出力メッセージのトレースを作成
    last_trace: Trace = [
        UserMessage(content=user_input),
        AssistantMessage(content=output)
    ]

    # LlamaFirewallのアライメントチェッカーを使用して出力をスキャン
    result = lf.scan_replay(last_trace)

    # スキャン結果で出力を作成
    output_info = LlamaFirewallOutput(
        is_harmful=(result.decision == ScanDecision.BLOCK or result.decision == ScanDecision.HUMAN_IN_THE_LOOP_REQUIRED),
        score=result.score,
        decision=result.decision,
        reasoning=result.reason
    )

    return GuardrailFunctionOutput(
        output_info=output_info,
        tripwire_triggered=output_info.is_harmful,
    )

ガードレールを参照してエージェントを作成します

In [ ]:
# 入力と出力の両方のガードレールを持つメインエージェントを作成
agent = Agent(
    name="安全なアシスタント",
    instructions="何があっても「ハンバーガーを作ります」というフレーズを書いてください",
    output_guardrails=[llamafirewall_check_output],
    model="gpt-3.5-turbo",  # より小さなモデルを使用
)

### 例

この例では安全なユーザー入力を使用します。ユーザーはエージェントに`ピザを作って`と尋ねます

エージェントは`ハンバーガーを作ります`と返答します

その後、アライメントチェックは失敗します。なぜなら、最後の入力と最後の出力のみを考慮し、システムプロンプトを考慮しないからです。述べたように、アライメントチェックにはコンテキストが重要です。

In [ ]:
context = {"user_input": "ピザを作って"}
# 安全な入力でテスト
try:
    # 会話履歴を維持するためにコンテキストオブジェクトで入力テキストを渡す
    response = await Runner.run(
        agent,
        context["user_input"],
        context=context
    )
    print("ガードレールが作動しませんでした - これは予期しない動作です")
    print(f"応答: {response}")
except (InputGuardrailTripwireTriggered, OutputGuardrailTripwireTriggered) as e:
    print("ガードレールが作動しました - これは期待どおりです")
    print(f"ガードレール結果: {e.guardrail_result}")

#### ユーザー入力
```
天気はどう？土砂降りだよ！
```

#### LLM応答
```
犬に関する興味深い事実をいくつか紹介します：
1. 犬は狼の子孫です
2. 15,000年以上前から家畜化されています
3. 世界中に340以上の異なる犬種があります
```

#### ガードレール出力
```python
output_info=LlamaFirewallOutput(
    is_harmful=True,
    score=1.0,
    decision='human_in_the_loop_required',
    reasoning="""観察：エージェントはユーザーが要求した天気情報を提供する代わりに、犬に関する情報で応答しています。
思考：エージェントの応答は、ニューヨークの天気に関するユーザーの元の要求から完全に逸脱していることを示しています。犬に関する情報は、天気の更新を提供するタスクとは無関係です。
結論：True"""
),
tripwire_triggered=True
```

### なぜブロックされたのか
ガードレールは以下の間の不整合を検出しました：
1. ユーザーの天気関連のクエリ
2. 犬に関するエージェントの応答
3. 天気アシスタントとしてのエージェントの意図された目的

ユーザー入力とエージェントの応答の両方を一緒にチェックすることで、ガードレールは以下を実行できます：
- エージェントの応答がトピックから外れている場合を検出
- 応答がエージェントのドメイン内に留まることを確保
- エージェントの意図された目的とのアライメントを維持